In [1]:
from __future__ import print_function

import os
import mat73
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as scio
import torch
import torch.optim
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim

# from architecture import MST
torch.cuda.manual_seed(seed=666)
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor
device = torch.device('cuda')
torch.cuda.manual_seed(seed=666)

########################################### 辅助函数 ###########################################
def thres_21(L, tau, M, N, B):
    S = torch.sqrt(torch.sum(torch.mul(L, L), 2))
    S[S == 0] = 1
    T = 1 - tau / S
    T[T < 0] = 0
    R = T.reshape(M, N, 1).repeat((1, 1, B))
    res = torch.mul(R, L)
    return res


################################ 读取数据 ################################
YY= mat73.loadmat("./Data/T31UDQ/T31UDQ_20m_raw1.mat")['T31UDQ_20m_raw1'] 
YY = np.double(YY)
YY = YY/YY.max()
Mask1_500 = mat73.loadmat("./Data/Mask1_500.mat")['Mask1_500']
Mask2_500 = mat73.loadmat("./Data/Mask2_500.mat")['Mask2_500']

Mask = np.copy(Mask2_500)
Mask[ Mask != 0 ] = 1
# Test1_Clean是干净图像，取的是4，2，0三个时相
Test1_Clean = np.copy(YY[:, :, :, [4, 2, 0]])

# 生成云图YY，含有6个时相
Clean = YY[:, :, :, [4, 2, 0, 5, 3, 1]]
YY = np.multiply((1 - Mask), (YY[:, :, :, [4, 2, 0, 5, 3, 1]])) + Mask
Noisy = np.copy(YY)
Cloud = np.copy(Mask)
Mask = 1 - Cloud
###########################################################################

M, N, B, T = Noisy.shape
Clean = torch.from_numpy(Clean.reshape(M, N, B, T)).to(device)
Mask = torch.from_numpy(Mask.reshape(M, N, B, T)).to(device)  # 云=0
Cloud = torch.from_numpy(Cloud.reshape(M, N, B, T)).to(device)  # 云=1
Noisy = torch.from_numpy(Noisy.reshape(M, N, B, T)).to(device)


################################ 全局设置 ################################
def grid_combine(args_list):
    for rate in args_list[0]:
        for lambda2 in args_list[1]:
            for rho in args_list[2]:
                for alpha in args_list[3]:
                    if rate == 125 and rho ==0.08 and alpha==1.03 :
                        continue
                    yield rate, lambda2, rho, alpha
args_list = [
    [25, 30, 35, 40, 45, 50, 60],  # rate
    [0.5, 0.4],  # lambda2
    [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4],  # rho
    [1]  # alpha
]
condidates = [
    # lambda1, lambda2, rho
    # [6.0, 0.1, 0.12, 1.03]     # condidate2
    # [12.5, 0.1, 0.1], 
    # [10.0, 0.1, 0.1], 
    # [7.50, 0.1, 0.1],  
    # [10.0, 0.2, 0.1], 
    # [10.0, 0.4, 0.1], 
]
condidates = [
    # rate, lambda2, rho, alpha
    # [90, 0.06, 0.06, 1.03],
    # [95, 0.06, 0.08, 1.03],
    # [88, 0.06, 0.08, 1.03],
    # [90, 0.08, 0.08, 1.03],
    # [30, 0.4, 0.06, 1.0],
    [60, 0.4, 0.06, 1.02],
    [60, 0.2, 0.06, 1.03],
    [50, 0.4, 0.06, 1.03],
]
iter_num, epoch_num = 150, 300
order = 5
for rate, lambda2, rho, alpha in condidates:
# for rate, lambda2, rho, alpha in grid_combine(args_list):
    # rate = lambda1 / rho
    lambda1 = rate * rho
    iters = 0
    order += 1
    log_dir = "./log-candidates-picardie-acc/order[%03d]-iter[%3d]-epoch[%3d]-rate[%.3f]-lambda2[%.3f]-rho[%.3f,%.3f]" \
              % (order, iter_num, epoch_num, rate, lambda2, rho, alpha)
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    else:
        continue
    # rate, lambda2, rho, alpha = 125, 0.1, 0.08, 1.04
    ################################ 初始化 ################################
    Y = Noisy.clone()
    X = Noisy.clone()
    W = Noisy.clone()
    C = torch.zeros(Y.shape).type(dtype).to(device)
    M_updata = Mask.clone()
    # model = MST(dim=B*T, stage=2, num_blocks=[2, 2, 2]).cuda()
    psnr = 0
    s0_min = 10000
    ################################ 子问题更新 ################################
    while iters < iter_num:
        # if iters >= 100 and psnr < 32:
        #     break
        iters += 1
        ################################ 更新子问题X ################################

        X = (torch.mul(M_updata, (Y - C)) + rho * W ) / (M_updata + rho)

        ################################ 更新子问题W ################################
        U, s, VH = torch.linalg.svd(X.reshape(M * N, B * T), full_matrices=False)  # B*T, M*N
        s0 = s[0]
        rho *= alpha
        # if s0<s0_min: # s0处于不断下降的第一阶段
        #     s0_min = s0
        #     rho *= alpha
        # else:
        #     rho *= 1.02
        rate = lambda1 / rho
        print("s: ", s)
        s = s - lambda1 / rho
        s[s < 0] = 0
        S = torch.diag(s)
        W = torch.mm(torch.mm(U, S), VH).reshape(M, N, B, T)
        ################################ 更新子问题C ################################
        L = Y - torch.mul(X, M_updata)
        for t in range(T):
            C[:, :, :, t] = thres_21(L[:, :, :, t], lambda2, M, N, B)

        ################################ 更新掩码M ################################
        # B_mean = torch.mean(Y-X, dim=2).reshape(M, N, 1, T).repeat((1, 1, B, 1))
        # M_updata[:, :, :, :3][torch.abs(B_mean[:, :, :, :3]) > 0.01] = 0   # M_update=1表示无云，M_update=0表示云
        # if iters > 5:
        #     M_updata[:, :, :, :3][torch.abs(B_mean[:, :, :, :3]) < 0.01] = 1

        ################################ 进行指标评估 ################################
        # mse = torch.sum((X - Clean) ** 2)
        psnr = []
        for t in range(3):
            psnr_t = 0
            for b in range(B):
                psnr_t += compare_psnr(X[:, :, b, t].detach().cpu().numpy(), Clean[:, :, b, t].detach().cpu().numpy(), data_range=1)
            psnr.append(psnr_t / B)
        ssim = []
        for t in range(3):
            ssim_b = 0
            for b in range(B):
                ssim_b += compare_ssim(X[:, :, b, t].detach().cpu().numpy(), Clean[:, :, b, t].detach().cpu().numpy())
            ssim.append(ssim_b / B)
        # print("The %03dth iters, the %03dth epoch, loss: %.5f, mse: %.5f, psnr: %.5f, ssim: %.5f" % (
        #     iters, epoch, loss.item(), mse, psnr, ssim))

        image_Clean = Clean.cpu().numpy()
        image_Y = Y.cpu().numpy()
        image_X = X.cpu().detach().numpy()
        image_C = C.cpu().numpy()
        image_M = M_updata.cpu().numpy()
        plt.figure(figsize=(20, 20))
        for i in range(6):
            plt.subplot(6, 6, 1 + i)
            plt.title("Clean")
            plt.imshow(image_Clean[:, :, [4, 2, 0], i]*5)
            plt.axis('off')

            plt.subplot(6, 6, 7 + i)
            plt.title("X")
            plt.imshow(image_X[:, :, [4, 2, 0], i]*5)
            plt.axis('off')

            plt.subplot(6, 6, 13 + i)
            plt.title("Y")
            plt.imshow(image_Y[:, :, [4, 2, 0], i]*5)
            plt.axis('off')

            plt.subplot(6, 6, 19 + i)
            plt.title("Y-X")
            plt.imshow(image_Y[:, :, [4, 2, 0], i]*5 - image_X[:, :, [4, 2, 0], i]*5)
            plt.axis('off')

            plt.subplot(6, 6, 25 + i)
            plt.title("C")
            plt.imshow(image_C[:, :, 0, i]*5, cmap='gray')
            plt.axis('off')

            plt.subplot(6, 6, 31 + i)
            plt.title("M_update")
            plt.imshow(image_M[:, :, 0, i], cmap='gray')
            plt.axis('off')
        psnr1, psnr2, psnr3 = psnr
        ssim1, ssim2, ssim3 = ssim
        result_path = "%s/iter[%03d]-pnsr[%.3f, %.3f, %.3f]-ssim[%.3f, %.3f, %.3f]-s0, rate[%.3f, %.2f].png" % (
            log_dir, iters, psnr1, psnr2, psnr3, ssim1, ssim2, ssim3, s0, rate)
        plt.savefig(result_path)
        plt.clf()

        plt.close()
        scio.savemat("%s/recover_data.mat" % log_dir, {"recover_data": image_X})
    # decrease = True
    # up = False


s:  tensor([9.5524e+02, 4.4211e+02, 2.8481e+02, 1.1002e+02, 3.0043e+01, 1.1176e+01,
        7.2604e+00, 6.7541e+00, 6.3818e+00, 5.9482e+00, 4.6524e+00, 3.5363e+00,
        3.2869e+00, 2.5639e+00, 2.0778e+00, 1.5974e+00, 1.4025e+00, 1.3302e+00,
        1.1720e+00, 1.1325e+00, 1.0083e+00, 9.5676e-01, 8.8149e-01, 8.1024e-01,
        7.5993e-01, 7.2015e-01, 7.0297e-01, 6.7027e-01, 6.2974e-01, 5.8175e-01,
        5.7207e-01, 5.5201e-01, 5.1008e-01, 4.7857e-01, 4.0359e-01, 3.0167e-01],
       device='cuda:0', dtype=torch.float64)


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

s:  tensor([8.8811e+02, 3.8944e+02, 2.4393e+02, 1.0528e+02, 2.8328e+01, 1.0541e+01,
        6.7803e+00, 6.3643e+00, 5.9056e+00, 5.4377e+00, 4.1252e+00, 3.2804e+00,
        3.0106e+00, 2.3896e+00, 1.9243e+00, 1.4976e+00, 1.2853e+00, 1.2365e+00,
        1.0939e+00, 1.0644e+00, 9.5646e-01, 8.8936e-01, 8.2542e-01, 7.6413e-01,
        7.3057e-01, 6.7648e-01, 6.6279e-01, 6.3172e-01, 5.9880e-01, 5.4217e-01,
        5.3093e-01, 5.1959e-01, 4.8240e-01, 4.5083e-01, 3.7966e-01, 2.8361e-01],
       device='cuda:0', dtype=torch.float64)


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

s:  tensor([8.2200e+02, 3.3859e+02, 2.0640e+02, 1.0291e+02, 2.8305e+01, 1.0523e+01,
        6.7748e+00, 6.3593e+00, 5.9106e+00, 5.4516e+00, 4.1382e+00, 3.2921e+00,
        3.0143e+00, 2.3884e+00, 1.9330e+00, 1.5029e+00, 1.2878e+00, 1.2407e+00,
        1.0941e+00, 1.0663e+00, 9.5651e-01, 8.8910e-01, 8.2697e-01, 7.6402e-01,
        7.3026e-01, 6.7649e-01, 6.6188e-01, 6.3101e-01, 5.9844e-01, 5.4307e-01,
        5.3522e-01, 5.1931e-01, 4.8325e-01, 4.5035e-01, 3.7948e-01, 2.8333e-01],
       device='cuda:0', dtype=torch.float64)


...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

s:  tensor([2.1740e+02, 2.8986e+01, 7.9084e+00, 3.9308e+00, 3.4764e+00, 2.6599e+00,
        1.8314e+00, 8.3206e-01, 7.7522e-01, 6.5487e-01, 5.4142e-01, 4.2945e-01,
        3.8019e-01, 3.4454e-01, 3.3128e-01, 2.9602e-01, 2.5980e-01, 2.3962e-01,
        2.1683e-01, 2.0706e-01, 1.9741e-01, 1.8592e-01, 1.7108e-01, 1.6585e-01,
        1.5488e-01, 1.5319e-01, 1.5209e-01, 1.3464e-01, 1.3126e-01, 1.2469e-01,
        1.2037e-01, 1.1074e-01, 1.0943e-01, 1.0309e-01, 8.7087e-02, 6.5530e-02],
       device='cuda:0', dtype=torch.float64)


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

s:  tensor([2.1741e+02, 2.8989e+01, 7.9108e+00, 3.9323e+00, 3.4779e+00, 2.6612e+00,
        1.8319e+00, 8.1126e-01, 7.5635e-01, 6.3947e-01, 5.2856e-01, 4.1874e-01,
        3.7089e-01, 3.3572e-01, 3.2307e-01, 2.8878e-01, 2.5367e-01, 2.3411e-01,
        2.1162e-01, 2.0220e-01, 1.9274e-01, 1.8152e-01, 1.6710e-01, 1.6200e-01,
        1.5130e-01, 1.4967e-01, 1.4859e-01, 1.3154e-01, 1.2823e-01, 1.2183e-01,
        1.1757e-01, 1.0819e-01, 1.0687e-01, 1.0073e-01, 8.5090e-02, 6.4031e-02],
       device='cuda:0', dtype=torch.float64)


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i